# Zadanie 1 – Rozpakowanie ZIP i liczenie plików/folderów

In [52]:

import os
import zipfile
import pandas as pd
import sys

# Ścieżka do pliku ZIP
zip_path = 'dane/test.zip'
extract_path = 'dane'

# Rozpakuj ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Licz foldery i pliki w rozpakowanym katalogu
base_folder = os.path.join(extract_path, 'test')
folder_count = 0
file_count = 0

for root, dirs, files in os.walk(base_folder):
    # Pomijamy katalog główny
    if root != base_folder:
        folder_count += len(dirs)
        file_count += len(files)

print(f"[Zadanie 1] Liczba folderów: {folder_count}")
print(f"[Zadanie 1] Liczba plików: {file_count}")

[Zadanie 1] Liczba folderów: 2
[Zadanie 1] Liczba plików: 3


# Zadanie 2 – Funkcja file_stats

In [49]:
def file_stats(base_path, folder_name):
    for root, dirs, files in os.walk(base_path):
        if os.path.basename(root) == folder_name:
            total_size = 0
            file_count = 0
            for file in files:
                file_path = os.path.join(root, file)
                if os.path.isfile(file_path):
                    total_size += os.path.getsize(file_path)
                    file_count += 1
            return file_count, total_size
    return 0, 0

# Przykład użycia:
print(file_stats('dane/test', '3'))

(1, 0)


# Zadanie 3 – Funkcja fs_stats

In [46]:
def fs_stats(path):
    data = []
    for root, dirs, files in os.walk(path):
        for name in dirs + files:
            full_path = os.path.join(root, name)
            is_file = os.path.isfile(full_path)
            is_dir = os.path.isdir(full_path)
            size = os.path.getsize(full_path) if is_file else 0
            try:
                line_count = sum(1 for _ in open(full_path, encoding='utf-8')) if is_file else 0
            except:
                line_count = 0
            data.append({
                "nazwa": name,
                "ścieżka_względna": os.path.relpath(full_path, path),
                "ścieżka_bezwzględna": os.path.abspath(full_path),
                "czy_folder": is_dir,
                "czy_plik": is_file,
                "rozmiar_bajty": size,
                "liczba_linii": line_count
            })
    return pd.DataFrame(data)

# Przykład:
df = fs_stats('dane/test')
print(df.head())


                        nazwa              ścieżka_względna  \
0                           2                             2   
1                           1                             1   
2  Nowy dokument tekstowy.txt  2/Nowy dokument tekstowy.txt   
3                           3                           1/3   
4  Nowy dokument tekstowy.txt  1/Nowy dokument tekstowy.txt   

                               ścieżka_bezwzględna  czy_folder  czy_plik  \
0                             /content/dane/test/2        True     False   
1                             /content/dane/test/1        True     False   
2  /content/dane/test/2/Nowy dokument tekstowy.txt       False      True   
3                           /content/dane/test/1/3        True     False   
4  /content/dane/test/1/Nowy dokument tekstowy.txt       False      True   

   rozmiar_bajty  liczba_linii  
0              0             0  
1              0             0  
2              0             0  
3              0             0  

# Zadanie 4 – Scalanie zawartości plików do jednego CSV

In [45]:
def save_all_texts_to_csv(path, output_csv='scalone_dane.csv'):
    df = fs_stats(path)
    text_data = []

    for _, row in df[df['czy_plik']].iterrows():
        try:
            with open(row['ścieżka_bezwzględna'], 'r', encoding='utf-8') as f:
                content = f.read()
            text_data.append({
                'plik': row['ścieżka_względna'],
                'treść': content
            })
        except:
            continue  # pomiń pliki binarne lub z błędnym kodowaniem

    result_df = pd.DataFrame(text_data)
    result_df.to_csv(output_csv, index=False)
    print(f"[Zadanie 4] Zapisano {len(result_df)} plików do pliku {output_csv}")

# Przykład:
save_all_texts_to_csv('dane/test')

[Zadanie 4] Zapisano 3 plików do pliku scalone_dane.csv


# Zadanie 5 – Wyjaśnienie różnicy w getsizeof()

In [36]:
print("[Zadanie 5] Porównanie sys.getsizeof dla list")

a = []
b = [1]
c = [1, 2, 3]

print(f"Pusta lista: {sys.getsizeof(a)} bajtów")
print(f"Lista z 1 elementem: {sys.getsizeof(b)} bajtów")
print(f"Lista z 3 elementami: {sys.getsizeof(c)} bajtów")

print("Wyjaśnienie: sys.getsizeof() mierzy rozmiar obiektu w pamięci (nie zawartości).\nLista w Pythonie to dynamiczna tablica, która rezerwuje pamięć z nadmiarem.\nRozmiar rośnie skokowo przy dodawaniu elementów (np. x1.125 lub x2).\n")

[Zadanie 5] Porównanie sys.getsizeof dla list
Pusta lista: 56 bajtów
Lista z 1 elementem: 64 bajtów
Lista z 3 elementami: 88 bajtów
Wyjaśnienie: sys.getsizeof() mierzy rozmiar obiektu w pamięci (nie zawartości).
Lista w Pythonie to dynamiczna tablica, która rezerwuje pamięć z nadmiarem.
Rozmiar rośnie skokowo przy dodawaniu elementów (np. x1.125 lub x2).

